In [ ]:
import polars as pl
from model import UserBasedCF, ItemBasedCF, MatrixFactorizationCF

In [ ]:
# Directory paths
parquet_dir = 'data/parquets'
json_path = 'data/game_details/SteamTopSellers_game_details.json'
selection_file = None

train_data, _ = load_data(parquet_dir, json_path, selection_file)

In [ ]:
# Train the models on the training data
def train_model(train_data, selection_name="")
    model_dir = 'data/model'
    
    user_based_model = UserBasedCF()
    user_based_model.fit(train_data)
    user_based_model.save(f'{model_dir}/user_based_cf_model{model_name}.pkl')
    
    item_based_model = ItemBasedCF()
    item_based_model.fit(train_data)
    item_based_model.save(f'{model_dir}/item_based_cf_model{model_name}.pkl')
    
    matrix_factorization_model = MatrixFactorizationCF()
    matrix_factorization_model.fit(train_data)
    matrix_factorization_model.save(f'{model_dir}/matrix_factorization_cf_model{model_name}.pkl')

In [ ]:
train_model(train_data)